In [2]:
! ls Datasets/new_test

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix 
import itertools

In [3]:
#DATADIR = r'D:\EDU Files\project\fire\MNet_Vgg Fire tuning\BowFire_Data'
DATADIR = r'D:\EDU Files\project\fire\MNet_Vgg Fire tuning\data_without_BoWFire_With_NSDataset\test'

CATEGORIES = ['Fire', 'Non_Fire']

In [4]:
IMG_SIZE = 64
def create_training_data():
    training_data = []
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=C 1=O

        for img in tqdm(os.listdir(path)):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
              
    return training_data

In [5]:
training_data = create_training_data()


100%|████████████████████████████████████████████████████████████████████████████| 28903/28903 [10:13<00:00, 47.11it/s]


In [6]:
import random
test_image_num=58704
print(len(training_data))
random.shuffle(training_data)
test_labels=np.zeros((test_image_num,1))

c=0
for sample in training_data:
    test_labels[c]=(sample[1])
    c+=1
print(c)
actual_labels=(test_labels.reshape(test_image_num,))
print(actual_labels.shape)
actual_labels.astype(int)

58704
58704
(58704,)


array([1, 1, 1, ..., 1, 0, 0])

In [7]:
X = []
Y = []

for features,label in training_data:
    X.append(features)
    Y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0
X.shape[1:]

(64, 64, 3)

In [8]:
#import tensorflow as tf
from keras.models import load_model
#from keras.preprocessing.image import img_to_array
#import numpy as np
#import cv2
#import time


model=load_model('D:\EDU Files\project\fire\MNet_Vgg Fire tuning\AJAK_Work_Most_Recent_Work\Omama_Work\Fire-64x64_new_train1_our.h5')

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


OSError: Unable to open file (unable to open file: name = 'D:\EDU Files\projectire\MNet_Vgg Fire tuning\AJAK_Work_Most_Recent_Work\Omama_Work\Fire-64x64_new_train1_our.h5', errno = 22, error message = 'Invalid argument', flags = 0, o_flags = 0)

In [ ]:
predicted_labels=model.predict_classes(X)
predicted_labels=(predicted_labels.reshape(test_image_num,))
#predicted_labels = np.array(predicted_labels) 
#print(predicted_labels.shape)
predicted_labels.astype(int)


In [ ]:
'''res = model.predict(X).tolist()
for i in range(len(res)):
    res[i] = res[i][0]
#print(res)
#len(res)'''

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

cm = confusion_matrix(actual_labels, predicted_labels)
#test_batches.class_indices
cm_plot_labels=['Fire','No Fire']
plot_confusion_matrix(cm, cm_plot_labels,title='Confusion Matrix')

In [ ]:
tp=cm[0][0]
fn=cm[0][1]
fp=cm[1][0]
tn=cm[1][1]
print("tp"+' '+str(tp))
print("fn"+' '+str(fn))
print("fp"+' '+str(fp))
print("tn"+' '+str(tn))

In [ ]:
Recall=tp/(tp+fn)
Precision=tp/(tp+fp)
f_measure= 2*((Precision*Recall)/(Precision+Recall))

print(Precision, Recall, f_measure)



In [ ]:
model.evaluate(X, Y)

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15874664629625462505
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3227792179
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12940274704012081081
physical_device_desc: "device: 0, name: GeForce 940M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]
